In [10]:
import yfinance as yf
import talib
import numpy as np
import random
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)  # Ignore DeprecationWarning
warnings.filterwarnings('ignore', category=FutureWarning) 

# Function to fetch historical stock data using yfinance
def fetch_stock_data(ticker, start_date, end_date):
    try:
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        return stock_data
    except: 
        print("error in fetching data")
# Function to calculate MACD using TA-Lib
def calculate_macd(data,fast_period,slow_period):
    try:
        macd, signal, _ = talib.MACD(data['Close'], fastperiod=fast_period, slowperiod=slow_period, signalperiod=9)
        return macd, signal
    except: 
        print("error in calculatin macd signal")
# Function to calculate Rate of Change (ROC) using TA-Lib
def calculate_roc(data, period):
    try:
        roc = talib.ROC(data['Close'], timeperiod=period)
        return roc
    except:
        print("this period caused the error",period)


# Function to calculate the derivative of RSI
# Function to calculate the derivative of RSI
def calculate_rsi_derivative(data, period):
    try:
        if len(data) >= period:
            rsi = talib.RSI(data['Close'], timeperiod=period)
            rsi_derivative = data.Close.diff()
            return rsi_derivative
        else:
            print("Insufficient data for RSI derivative calculation")
            return None
    except Exception as e:
        print(f"Error calculating RSI derivative: {e}")
        print(period)
        return None
# Function to simulate a trailing stop-loss exit condition
def trailing_stop_loss(data, stop_loss_percentage):
    highest_high = data['High'].rolling(window=20).max()
    stop_loss = highest_high * (1 - stop_loss_percentage)
    return stop_loss

In [40]:
def evaluate_strategy(data, fast_period, slow_period, roc_period, rsi_period, roc_threshold, rsi_derivative_threshold, stop_loss_percentage):
    # Calculate indicators
    print(fast_period,slow_period,roc_period, rsi_period, roc_threshold, rsi_derivative_threshold, stop_loss_percentage)
    data['MACD'], data['Signal'] = calculate_macd(data, fast_period, slow_period)
    data['ROC'] = calculate_roc(data, roc_period)
    data['RSI_Derivative'] = calculate_rsi_derivative(data, rsi_period)
    
    # Apply trailing stop-loss exit condition
    stop_loss = trailing_stop_loss(data, stop_loss_percentage)
    
    # Simulated trading logic
    position = 0  # 0 for no position, 1 for long position
    profit = 0
    total_trades = 0
    
    for i in range(1, len(data)):
        try:
            if data['MACD'][i] > data['Signal'][i] :#and data['ROC'][i] > roc_threshold and data['RSI_Derivative'][i] > rsi_derivative_threshold:
                if position == 0:
                    position = 1  # Buy (enter long position)
                    entry_price = data['Close'][i]
                    stop_loss_price = entry_price - (entry_price * 0.01* stop_loss_percentage)
                    print(f"Entry: {entry_price} on {data.index[i]} Stop Loss : {stop_loss_price}")
                    total_trades += 1  
            elif position == 1:
                if data['Low'][i] < stop_loss_price:
                    position = 0  # Sell due to stop loss
                    exit_price = stop_loss_price
                    profit += exit_price - entry_price  # Calculate profit/loss
                    print(f"Exit: {exit_price} on {data.index[i]} exited on stop loss")
                    print(f"Profit: {profit}")
#                 elif data['MACD'][i] < data['Signal'][i] or data['ROC'][i] < roc_threshold or data['RSI_Derivative'][i] < rsi_derivative_threshold:
#                     position = 0  # Sell due to exit condition
#                     exit_price = data['Close'][i]
#                     profit += exit_price - entry_price  # Calculate profit/loss
#                     print(f"Exit: {exit_price} on {data.index[i]} exited on MACD condition")
#                     print(f"Profit: {profit}")
        except:
            print("something caused the error")
    
    return profit

In [41]:
# Fetch historical stock data
ticker_symbol = '^NSEI'  # Replace with your desired stock symbol
start_date = '2021-01-01'
end_date = '2021-12-31'
stock_data = fetch_stock_data(ticker_symbol, start_date, end_date)

# Genetic Algorithm parameters
population_size = 100
num_generations = 10
mutation_rate = 0.1
tournament_size = 5

# GA: Initialization
def generate_individual():
    fast_p = random.randint(2,10)
    slow_p = random.randint(fast_p,20)
    return [fast_p,slow_p, random.randint(1, 7), random.randint(2, 7), random.randint(1, 30), random.randint(1, 30), random.randint(1, 3)]

def crossover(parent1, parent2):
    crossover_point = random.randint(0, len(parent1) - 1)
    child = parent1[:crossover_point] + parent2[crossover_point:]
    return child

def mutate(individual):
    gene_to_mutate = random.randint(0, len(individual) - 1)
    if gene_to_mutate < 2:
#         individual[gene_to_mutate] = random.randint(5, 200)
        pass
    else:
        individual[gene_to_mutate] = random.randint(1, 5) if gene_to_mutate == 5 else random.randint(1, 30)
    return individual
# Known or useful parameter values
known_parameters = [
    [3, 15, 5, 5, 15, 20, 2],  # Example set of known parameters
    # Add more known parameter sets if available
]

# Randomly generate the rest of the population
remaining_population = [generate_individual() for _ in range(population_size - len(known_parameters))]

# Initialize population using known parameter values and randomly generated values
population = known_parameters + remaining_population
# population = [generate_individual() for _ in range(population_size)]

# GA: Evolution process
for generation in range(num_generations):
    fitness_scores = [evaluate_strategy(stock_data.copy(), *individual) for individual in population]
    sorted_population = [x for _, x in sorted(zip(fitness_scores, population), key=lambda pair: pair[0], reverse=True)]
    
    new_population = []
    for _ in range(population_size):
        tournament_indices = random.sample(range(population_size), tournament_size)
        tournament_contestants = [sorted_population[i] for i in tournament_indices]
        parent1, parent2 = random.sample(tournament_contestants, 2)  # Select two parents
        
        child1 = crossover(parent1, parent2)  # Apply crossover
        child2 = crossover(parent2, parent1)  # Apply crossover with parents swapped
        
        # Perform mutation on children
        if random.random() < mutation_rate:
            child1 = mutate(child1)
        if random.random() < mutation_rate:
            child2 = mutate(child2)
        
        new_population.extend([child1, child2])  # Add children to the new population
    
    population = new_population[:population_size]  # Select the best individuals to form the next generation
        
#         winner = max(tournament_contestants, key=lambda x: evaluate_strategy(stock_data.copy(), *x))
        
#         if random.random() < mutation_rate:
#             winner = mutate(winner)
        
#         new_population.append(winner)
    
#     population = new_population

# Find the best individual (parameters) from the final population
best_individual_index = np.argmax(fitness_scores)
best_individual = population[best_individual_index]
best_fitness_score = fitness_scores[best_individual_index]

print("Best Parameters:", best_individual)
print("Best Fitness Score:", best_fitness_score)


[*********************100%%**********************]  1 of 1 completed
3 15 5 5 15 20 2
Entry: 14789.9501953125 on 2021-02-03 00:00:00 Stop Loss : 14494.15119140625
Exit: 14494.15119140625 on 2021-02-26 00:00:00 exited on stop loss
Profit: -295.7990039062497
Entry: 15245.599609375 on 2021-03-03 00:00:00 Stop Loss : 14940.6876171875
Exit: 14940.6876171875 on 2021-03-08 00:00:00 exited on stop loss
Profit: -600.7109960937505
Entry: 15098.400390625 on 2021-03-09 00:00:00 Stop Loss : 14796.4323828125
Exit: 14796.4323828125 on 2021-03-15 00:00:00 exited on stop loss
Profit: -902.6790039062507
Entry: 14845.099609375 on 2021-03-30 00:00:00 Stop Loss : 14548.1976171875
Exit: 14548.1976171875 on 2021-04-12 00:00:00 exited on stop loss
Profit: -1199.5809960937513
Entry: 14485.0 on 2021-04-26 00:00:00 Stop Loss : 14195.3
3 18 4 2 12 16 1
Entry: 15115.7998046875 on 2021-02-08 00:00:00 Stop Loss : 14964.641806640626
Exit: 14964.641806640626 on 2021-02-19 00:00:00 exited on stop loss
Profit: -151.1579

Entry: 15245.599609375 on 2021-03-03 00:00:00 Stop Loss : 14788.23162109375
Exit: 14788.23162109375 on 2021-03-15 00:00:00 exited on stop loss
Profit: -910.8419824218745
Entry: 14845.099609375 on 2021-03-30 00:00:00 Stop Loss : 14399.74662109375
Exit: 14399.74662109375 on 2021-04-12 00:00:00 exited on stop loss
Profit: -1356.1949707031254
Entry: 14653.0498046875 on 2021-04-27 00:00:00 Stop Loss : 14213.458310546876
6 18 7 5 15 14 2
Entry: 15115.7998046875 on 2021-02-08 00:00:00 Stop Loss : 14813.48380859375
Exit: 14813.48380859375 on 2021-02-22 00:00:00 exited on stop loss
Profit: -302.3159960937501
Entry: 15245.599609375 on 2021-03-03 00:00:00 Stop Loss : 14940.6876171875
Exit: 14940.6876171875 on 2021-03-05 00:00:00 exited on stop loss
Profit: -607.2279882812509
Entry: 15098.400390625 on 2021-03-09 00:00:00 Stop Loss : 14796.4323828125
Exit: 14796.4323828125 on 2021-03-15 00:00:00 exited on stop loss
Profit: -909.1959960937511
Entry: 14690.7001953125 on 2021-03-31 00:00:00 Stop Loss 

5 5 7 5 5 26 2
7 9 1 5 26 12 3
Entry: 14647.849609375 on 2021-02-02 00:00:00 Stop Loss : 14208.41412109375
Exit: 14208.41412109375 on 2021-04-19 00:00:00 exited on stop loss
Profit: -439.4354882812495
Entry: 14485.0 on 2021-04-26 00:00:00 Stop Loss : 14050.45
9 12 6 6 6 23 3
Entry: 14789.9501953125 on 2021-02-03 00:00:00 Stop Loss : 14346.251689453125
Exit: 14346.251689453125 on 2021-03-25 00:00:00 exited on stop loss
Profit: -443.69850585937456
Entry: 14690.7001953125 on 2021-03-31 00:00:00 Stop Loss : 14249.979189453124
Exit: 14249.979189453124 on 2021-04-12 00:00:00 exited on stop loss
Profit: -884.4195117187501
Entry: 14653.0498046875 on 2021-04-27 00:00:00 Stop Loss : 14213.458310546876
3 20 1 7 6 4 2
Entry: 15106.5 on 2021-02-10 00:00:00 Stop Loss : 14804.37
Exit: 14804.37 on 2021-02-22 00:00:00 exited on stop loss
Profit: -302.1299999999992
Entry: 15245.599609375 on 2021-03-03 00:00:00 Stop Loss : 14940.6876171875
Exit: 14940.6876171875 on 2021-03-08 00:00:00 exited on stop loss

8 18 6 6 15 30 1
Entry: 15115.7998046875 on 2021-02-08 00:00:00 Stop Loss : 14964.641806640626
Exit: 14964.641806640626 on 2021-02-19 00:00:00 exited on stop loss
Profit: -151.15799804687413
Entry: 15174.7998046875 on 2021-03-10 00:00:00 Stop Loss : 15023.051806640626
Exit: 15023.051806640626 on 2021-03-15 00:00:00 exited on stop loss
Profit: -302.9059960937484
Entry: 14867.349609375 on 2021-04-01 00:00:00 Stop Loss : 14718.67611328125
Exit: 14718.67611328125 on 2021-04-12 00:00:00 exited on stop loss
Profit: -451.57949218749854
Entry: 14653.0498046875 on 2021-04-27 00:00:00 Stop Loss : 14506.519306640625
8 13 1 4 2 23 3
Entry: 14789.9501953125 on 2021-02-03 00:00:00 Stop Loss : 14346.251689453125
Exit: 14346.251689453125 on 2021-03-25 00:00:00 exited on stop loss
Profit: -443.69850585937456
Entry: 14690.7001953125 on 2021-03-31 00:00:00 Stop Loss : 14249.979189453124
Exit: 14249.979189453124 on 2021-04-12 00:00:00 exited on stop loss
Profit: -884.4195117187501
Entry: 14653.0498046875 

7 16 1 4 12 28 3
Entry: 14895.650390625 on 2021-02-04 00:00:00 Stop Loss : 14448.78087890625
Exit: 14448.78087890625 on 2021-03-19 00:00:00 exited on stop loss
Profit: -446.8695117187508
Entry: 14690.7001953125 on 2021-03-31 00:00:00 Stop Loss : 14249.979189453124
Exit: 14249.979189453124 on 2021-04-12 00:00:00 exited on stop loss
Profit: -887.5905175781263
Entry: 14653.0498046875 on 2021-04-27 00:00:00 Stop Loss : 14213.458310546876
9 17 4 6 27 1 2
Entry: 14924.25 on 2021-02-05 00:00:00 Stop Loss : 14625.765
Exit: 14625.765 on 2021-02-24 00:00:00 exited on stop loss
Profit: -298.4850000000006
Entry: 15174.7998046875 on 2021-03-10 00:00:00 Stop Loss : 14871.30380859375
Exit: 14871.30380859375 on 2021-03-15 00:00:00 exited on stop loss
Profit: -601.980996093751
Entry: 14867.349609375 on 2021-04-01 00:00:00 Stop Loss : 14570.0026171875
Exit: 14570.0026171875 on 2021-04-12 00:00:00 exited on stop loss
Profit: -899.3279882812512
Entry: 14653.0498046875 on 2021-04-27 00:00:00 Stop Loss : 14

...

6 10 7 2 18 30 2
Entry: 14647.849609375 on 2021-02-02 00:00:00 Stop Loss : 14354.8926171875
Exit: 14354.8926171875 on 2021-03-19 00:00:00 exited on stop loss
Profit: -292.9569921875009
Entry: 14845.099609375 on 2021-03-30 00:00:00 Stop Loss : 14548.1976171875
Exit: 14548.1976171875 on 2021-04-12 00:00:00 exited on stop loss
Profit: -589.8589843750015
Entry: 14485.0 on 2021-04-26 00:00:00 Stop Loss : 14195.3
4 5 6 6 26 22 3
Entry: 14281.2001953125 on 2021-02-01 00:00:00 Stop Loss : 13852.764189453124
5 6 6 5 18 25 3
Entry: 14647.849609375 on 2021-02-02 00:00:00 Stop Loss : 14208.41412109375
Exit: 14208.41412109375 on 2021-04-19 00:00:00 exited on stop loss
Profit: -439.4354882812495
Entry: 14485.0 on 2021-04-26 00:00:00 Stop Loss : 14050.45
10 14 5 4 25 30 3
Entry: 14895.650390625 on 2021-02-04 00:00:00 Stop Loss : 14448.78087890625
Exit: 14448.78087890625 on 2021-03-19 00:00:00 exited on stop loss
Profit: -446.8695117187508
Entry: 14867.349609375 on 2021-04-01 00:00:00 Stop Loss : 1442

6 15 5 5 15 5 2
Entry: 14789.9501953125 on 2021-02-03 00:00:00 Stop Loss : 14494.15119140625
Exit: 14494.15119140625 on 2021-02-26 00:00:00 exited on stop loss
Profit: -295.7990039062497
Entry: 15245.599609375 on 2021-03-03 00:00:00 Stop Loss : 14940.6876171875
Exit: 14940.6876171875 on 2021-03-05 00:00:00 exited on stop loss
Profit: -600.7109960937505
Entry: 15098.400390625 on 2021-03-09 00:00:00 Stop Loss : 14796.4323828125
Exit: 14796.4323828125 on 2021-03-15 00:00:00 exited on stop loss
Profit: -902.6790039062507
Entry: 14845.099609375 on 2021-03-30 00:00:00 Stop Loss : 14548.1976171875
Exit: 14548.1976171875 on 2021-04-12 00:00:00 exited on stop loss
Profit: -1199.5809960937513
Entry: 14653.0498046875 on 2021-04-27 00:00:00 Stop Loss : 14359.98880859375
5 6 6 5 18 25 21
Entry: 14647.849609375 on 2021-02-02 00:00:00 Stop Loss : 11571.80119140625
5 6 6 5 18 25 3
Entry: 14647.849609375 on 2021-02-02 00:00:00 Stop Loss : 14208.41412109375
Exit: 14208.41412109375 on 2021-04-19 00:00:00

5 5 5 7 30 4 2
4 11 6 7 11 5 3
Entry: 14647.849609375 on 2021-02-02 00:00:00 Stop Loss : 14208.41412109375
Exit: 14208.41412109375 on 2021-04-19 00:00:00 exited on stop loss
Profit: -439.4354882812495
Entry: 14485.0 on 2021-04-26 00:00:00 Stop Loss : 14050.45
10 14 5 7 17 25 1
Entry: 14895.650390625 on 2021-02-04 00:00:00 Stop Loss : 14746.69388671875
Exit: 14746.69388671875 on 2021-02-22 00:00:00 exited on stop loss
Profit: -148.95650390625087
Entry: 15174.7998046875 on 2021-03-10 00:00:00 Stop Loss : 15023.051806640626
Exit: 15023.051806640626 on 2021-03-15 00:00:00 exited on stop loss
Profit: -300.70450195312515
Entry: 14867.349609375 on 2021-04-01 00:00:00 Stop Loss : 14718.67611328125
Exit: 14718.67611328125 on 2021-04-12 00:00:00 exited on stop loss
Profit: -449.3779980468753
Entry: 14653.0498046875 on 2021-04-27 00:00:00 Stop Loss : 14506.519306640625
5 5 7 4 17 25 1
2 13 1 2 18 30 22
Entry: 14647.849609375 on 2021-02-02 00:00:00 Stop Loss : 11425.3226953125
6 10 6 2 18 5 2
Entr

Entry: 15109.2998046875 on 2021-02-09 00:00:00 Stop Loss : 14958.206806640625
Exit: 14958.206806640625 on 2021-02-19 00:00:00 exited on stop loss
Profit: -151.09299804687544
Entry: 15080.75 on 2021-03-04 00:00:00 Stop Loss : 14929.9425
Exit: 14929.9425 on 2021-03-05 00:00:00 exited on stop loss
Profit: -301.90049804687624
Entry: 15174.7998046875 on 2021-03-10 00:00:00 Stop Loss : 15023.051806640626
Exit: 15023.051806640626 on 2021-03-15 00:00:00 exited on stop loss
Profit: -453.6484960937505
Entry: 14867.349609375 on 2021-04-01 00:00:00 Stop Loss : 14718.67611328125
Exit: 14718.67611328125 on 2021-04-12 00:00:00 exited on stop loss
Profit: -602.3219921875007
Entry: 14653.0498046875 on 2021-04-27 00:00:00 Stop Loss : 14506.519306640625
2 19 7 2 25 22 2
Entry: 15109.2998046875 on 2021-02-09 00:00:00 Stop Loss : 14807.11380859375
Exit: 14807.11380859375 on 2021-02-22 00:00:00 exited on stop loss
Profit: -302.1859960937509
Entry: 15097.349609375 on 2021-02-25 00:00:00 Stop Loss : 14795.402

Best Parameters: [9, 5, 7, 4, 23, 19, 2]
Best Fitness Score: 0


In [6]:
stock_data = yf.download("^NSEI", start='2022-01-01',end='2023-12-28')

[*********************100%%**********************]  1 of 1 completed


In [39]:
# Use the best parameters obtained from the GA
#"Best Parameters (Fast Period, Slow Period, ROC Period, RSI Period, ROC Threshold, RSI Derivative Threshold, Stop Loss %):"
best_parameters = [7, 12, 7, 2, 24, 25, 2]

# Apply the strategy on out-of-sample data using the best parameters
out_of_sample_profit = evaluate_strategy(stock_data.copy(), *best_parameters)

# Print or analyze the performance metrics
print("Out-of-sample profit:", out_of_sample_profit)

7 12 7 2 24 25 2
Entry: 14789.9501953125 on 2021-02-03 00:00:00 Stop Loss : 14494.15119140625
Exit: 15118.9501953125 on 2021-02-18 00:00:00 exited on MACD condition
Profit: 329.0
Entry: 15245.599609375 on 2021-03-03 00:00:00 Stop Loss : 14940.6876171875
Exit: 14940.6876171875 on 2021-03-08 00:00:00 exited on stop loss
Profit: 24.0880078124992
Entry: 15098.400390625 on 2021-03-09 00:00:00 Stop Loss : 14796.4323828125
Exit: 14796.4323828125 on 2021-03-15 00:00:00 exited on stop loss
Profit: -277.880000000001
Entry: 14845.099609375 on 2021-03-30 00:00:00 Stop Loss : 14548.1976171875
Exit: 14548.1976171875 on 2021-04-12 00:00:00 exited on stop loss
Profit: -574.7819921875016
Entry: 14653.0498046875 on 2021-04-27 00:00:00 Stop Loss : 14359.98880859375
Exit: 14677.7998046875 on 2021-05-14 00:00:00 exited on MACD condition
Profit: -550.0319921875016
Entry: 14923.150390625 on 2021-05-17 00:00:00 Stop Loss : 14624.6873828125
Exit: 15635.349609375 on 2021-06-09 00:00:00 exited on MACD condition


In [86]:
random.randint(5, 20)

12